In [1]:
import datetime

import numpy as np
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import seaborn as sns

from tst import Transformer
from tst.loss import OZELoss

from src.dataset import OzeDataset
from src.utils import compute_loss
from src.visualization import map_plot_function, plot_values_distribution, plot_error_distribution, plot_errors_threshold, plot_visual_sample
from collections import defaultdict
import os
import pandas as pd

In [2]:
# Training parameters
#DATASET_PATH = 'datasets/dataset_57M.npz'
BATCH_SIZE = 16
NUM_WORKERS = 0
LR = 2e-4
EPOCHS = 30

# Model parameters
d_model = 64 # Lattent dim
q = 8 # Query size
v = 8 # Value size
h = 8 # Number of heads
N = 4 # Number of encoder and decoder to stack
attention_size = 12 # Attention window size
dropout = 0.2 # Dropout rate
pe = None # Positional encoding
chunk_mode = None

d_input = 5 # From dataset
d_output = 1 # From dataset

# Config
sns.set()
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")

Using device cuda:1


In [4]:
# Load transformer with Adam optimizer and MSE loss function
net = Transformer(d_input, d_model, d_output, q, v, h, N, attention_size=attention_size, dropout=dropout, chunk_mode=chunk_mode, pe=pe).to(device)
optimizer = optim.Adam(net.parameters(), lr=LR)
loss_function = OZELoss(alpha=0.3)

In [5]:
save_directory = "rawData_Results/shuffleAll_guidedAttn/e3/dbp/"
data_directory = '/data/datasets/bioz/b2b/'
training_size = 0.8
test_dict = defaultdict(dict)
subjects = next(os.walk(data_directory))[1]
x_train = None
y_train = None
init_flag = True
for subject in subjects:
  bioz_lists = pd.read_pickle(data_directory + subject + "/list_data_time_100.pckl")
  new_bioz_list = list()
  for b in bioz_lists:
    tmp_zeros = np.zeros((100-len(b),5))
    pad_b = np.concatenate((b,tmp_zeros),axis=0)
    new_bioz_list.append(pad_b)
  bioz_lists = np.asarray(new_bioz_list)  
  bioz_labels = pd.read_pickle(data_directory + subject + "/labels_100.pckl")
  bioz_labels = bioz_labels.values[:,0]

  num_samples = len(bioz_lists)

  indices = np.arange(num_samples)
  np.random.shuffle(indices)
  bioz_lists = bioz_lists[indices]
  bioz_labels = bioz_labels[indices]
  if init_flag == True:
    init_flag = False
    x_train = bioz_lists[0:int(num_samples*training_size)]
    y_train = bioz_labels[0:int(num_samples*training_size)]
  else:
    x_train = np.concatenate((x_train, bioz_lists[0:int(num_samples*training_size)]))
    y_train = np.concatenate((y_train, bioz_labels[0:int(num_samples*training_size)]))
  test_dict[subject]['x_test'] = bioz_lists[int(num_samples*training_size):]
  test_dict[subject]['y_test'] = bioz_labels[int(num_samples*training_size):]

In [6]:
num_batches = int(len(x_train)/BATCH_SIZE)

In [7]:
model_save_path = f'models/model_{datetime.datetime.now().strftime("%Y_%m_%d__%H%M%S")}.pth'
val_loss_best = np.inf

# Prepare loss history
hist_loss = np.zeros(EPOCHS)
hist_loss_val = np.zeros(EPOCHS)
for idx_epoch in range(EPOCHS):
    running_loss = 0
    with tqdm(total=len(x_train), desc=f"[Epoch {idx_epoch+1:3d}/{EPOCHS}]") as pbar:
        #for idx_batch, (x, y) in enumerate(dataloader_train):
        for idx_batch in range(num_batches):
            x = x_train[int(BATCH_SIZE*idx_batch):int(BATCH_SIZE*idx_batch+BATCH_SIZE)]
            y = y_train[int(BATCH_SIZE*idx_batch):int(BATCH_SIZE*idx_batch+BATCH_SIZE)]
            x = torch.Tensor(x)
            y = np.reshape(y,(len(y),1))
            y = torch.Tensor(y)

            print('check')
            print(x.shape)
            print(y.shape)



            optimizer.zero_grad()

            # Propagate input
            netout = net(x.to(device))
            #print(netout[0])
            # Comupte loss
            loss = loss_function(y.to(device), netout)

            # Backpropage loss
            loss.backward()

            # Update weights
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix({'loss': running_loss/(idx_batch+1)})
            pbar.update(x.shape[0])
        
        train_loss = running_loss/len(dataloader_train)
        val_loss = compute_loss(net, dataloader_val, loss_function, device).item()
        pbar.set_postfix({'loss': train_loss, 'val_loss': val_loss})
        
        hist_loss[idx_epoch] = train_loss
        hist_loss_val[idx_epoch] = val_loss
        
        if val_loss < val_loss_best:
            val_loss_best = val_loss
            torch.save(net.state_dict(), model_save_path)
        
plt.plot(hist_loss, 'o-', label='train')
plt.plot(hist_loss_val, 'o-', label='val')
plt.legend()
print(f"model exported to {model_save_path} with loss {val_loss_best:5f}")

[Epoch   1/30]:   0%|          | 0/9301 [00:00<?, ?it/s]/home/grads/j/jmartinez0304/miniconda3/envs/myenv/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([16, 0])) that is different to the input size (torch.Size([16, 100, 0])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
[Epoch   1/30]:   0%|          | 0/9301 [00:00<?, ?it/s]

check
torch.Size([16, 100, 5])
torch.Size([16, 1])


RuntimeError: The size of tensor a (100) must match the size of tensor b (16) at non-singleton dimension 1